In [34]:
import re
import pandas as pd
from nltk import regexp_tokenize

In [11]:
assembly = '''
	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 10, 14	sdk_version 10, 14
	.globl	_super_duper_print      ## -- Begin function super_duper_print
	.p2align	4, 0x90
_super_duper_print:                     ## @super_duper_print
	.cfi_startproc
## %bb.0:
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset %rbp, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register %rbp
	subq	$16, %rsp
	movq	%rdi, -8(%rbp)
	movq	-8(%rbp), %rsi
	leaq	L_.str(%rip), %rdi
	movb	$0, %al
	callq	_printf
	movl	%eax, -12(%rbp)         ## 4-byte Spill
	addq	$16, %rsp
	popq	%rbp
	retq
	.cfi_endproc
                                        ## -- End function
	.globl	_main                   ## -- Begin function main
	.p2align	4, 0x90
_main:                                  ## @main
	.cfi_startproc
## %bb.0:
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset %rbp, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register %rbp
	subq	$16, %rsp
	movl	$0, -4(%rbp)
	leaq	L_.str.1(%rip), %rdi
	callq	_super_duper_print
	movl	$1, %eax
	addq	$16, %rsp
	popq	%rbp
	retq
	.cfi_endproc
                                        ## -- End function
	.section	__TEXT,__cstring,cstring_literals
L_.str:                                 ## @.str
	.asciz	"%s\n"

L_.str.1:                               ## @.str.1
	.asciz	"this is the coolest c program ever written\n"


.subsections_via_symbols

'''

In [14]:
assembly[:30]

'\n\t.section\t__TEXT,__text,regul'

In [16]:
instruction_set = pd.read_csv("x86_instruction_set.csv")

In [25]:
instruction_set.columns

Index(['Instruction', 'Opcode', 'Valid 64-bit', 'Valid 32-bit', 'Valid 16-bit',
       'Feature Flags', 'Operand 1', 'Operand 2', 'Operand 3', 'Operand 4',
       'Tuple Type', 'Description'],
      dtype='object')

In [32]:
cleaned_instructions = [i.split()[0] for i in instruction_set["Instruction"]]



In [33]:
print(len(set(cleaned_instructions)))
print(list(set(cleaned_instructions))[:30])

1309
['PXOR', 'FCOMIP', 'CMPXCHG', 'OUTSW', 'FSUBR', 'VPMULHW', 'XBEGIN', 'VPROLQ', 'VGATHERPF0DPD', 'VPSIGNB', 'PMULLD', 'ROUNDSD', 'VGETMANTPS', 'VPHSUBD', 'VPMULLD', 'PMAXUW', 'VPADDQ', 'VZEROALL', 'VPSUBUSW', 'MINPS', 'PSUBUSB', 'VPCMPEQB', 'STOSW', 'VREDUCESS', 'VFIXUPIMMSS', 'SHR', 'MOVBE', 'PAND', 'PABSW', 'CMPSD']


In [158]:
cleaned_instructions_lower = [instruction.lower() for instruction in set(cleaned_instructions)]
cleaned_instructions_lower[:15]

['pxor',
 'fcomip',
 'cmpxchg',
 'outsw',
 'fsubr',
 'vpmulhw',
 'xbegin',
 'vprolq',
 'vgatherpf0dpd',
 'vpsignb',
 'pmulld',
 'roundsd',
 'vgetmantps',
 'vphsubd',
 'vpmulld']

In [447]:
s = ".direective here that is not important \t\t\n .second direcitve here\t\t\n movq %rax, %rsp\t\n mov %rsp, %rax\t\n .third directive here\t\t\n"

In [465]:
s = ".direective_here_that_is_not_important_\t\t\n .second_direcitve_here\t\t\n movq -8(%rax), %rsp\t\n mov -16(%rsp), %rax\t\n .third directive here\t\t\n movq %rsp, %rax"




In [466]:
re.split("\%\w+,", s)

['.direective_here_that_is_not_important_\t\t\n .second_direcitve_here\t\t\n movq -8(%rax), %rsp\t\n mov -16(%rsp), %rax\t\n .third directive here\t\t\n movq ',
 ' %rax']

In [467]:
"(?<=(\(|\())|"
"(?<=(?=%)\()"
"(?<=(?=\))\w{2,4}"

'(?<=(?=\\))\\w{2,4}'

In [468]:
# split on a parentheses before %
regex.split("(?<=\()(?=\%)", s)

['.direective_here_that_is_not_important_\t\t\n .second_direcitve_here\t\t\n movq -8(',
 '%rax), %rsp\t\n mov -16(',
 '%rsp), %rax\t\n .third directive here\t\t\n movq %rsp, %rax']

In [475]:
# split on 2-5 word characters before a parentheses
regex.split("(?<=\%\w{2,5})(?=\))", s)

['.direective_here_that_is_not_important_\t\t\n .second_direcitve_here\t\t\n movq -8(%rax',
 '), %rsp\t\n mov -16(%rsp',
 '), %rax\t\n .third directive here\t\t\n movq %rsp, %rax']

In [470]:
# split on % then 2-5 word characters before a whitespace
regex.split("(?<=\%\w{2,5})(?=\s)", s)

['.direective_here_that_is_not_important_\t\t\n .second_direcitve_here\t\t\n movq -8(%rax), %rsp',
 '\t\n mov -16(%rsp), %rax',
 '\t\n .third directive here\t\t\n movq %rsp, %rax']

In [472]:
# split on whitespace before %\w{2,5}
regex.split("(?<=\s)(?=\%\w{2,5})", s)

['.direective_here_that_is_not_important_\t\t\n .second_direcitve_here\t\t\n movq -8(%rax), ',
 '%rsp\t\n mov -16(%rsp), ',
 '%rax\t\n .third directive here\t\t\n movq ',
 '%rsp, ',
 '%rax']

In [476]:
# split on %\w{2,5} before a comma 
regex.split("(?<=\%\w{2,5})(?=,)", s)

['.direective_here_that_is_not_important_\t\t\n .second_direcitve_here\t\t\n movq -8(%rax), %rsp\t\n mov -16(%rsp), %rax\t\n .third directive here\t\t\n movq %rsp',
 ', %rax']

In [478]:
# all the ones above combined
regex.split('(?<=\%\w{2,5})(?=,)|(?<=\s)(?=\%\w{2,5})|(?<=\%\w{2,5})(?=\s)|(?<=\%\w{2,5})(?=\))|(?<=\()(?=\%)', s)

['.direective_here_that_is_not_important_\t\t\n .second_direcitve_here\t\t\n movq -8(',
 '%rax',
 '), ',
 '%rsp',
 '\t\n mov -16(',
 '%rsp',
 '), ',
 '%rax',
 '\t\n .third directive here\t\t\n movq ',
 '%rsp',
 ', ',
 '%rax']

In [460]:
regex.split("(?<=\()(?=\%)|(?<=\w{2,4})(?=\))", s)

['.direective_here_that_is_not_important_\t\t\n .second_direcitve_here\t\t\n movq -8(',
 '%rax',
 '), %rsp\t\n mov -16(',
 '%rsp',
 '), %rax\t\n .third directive here\t\t\n']

In [ ]:
regex.split("(?<=\()(?=\%)|(?<=\w{2,4})(?=\))", s)

In [435]:
regex.split("(?=\()",s)

['.direective_here_that_is_not_important_\t\t\n .second_direcitve_here\t\t\n movq -8',
 '(%rax), %rsp\t\n mov -16',
 '($rsp), %rax\t\n .third directive here\t\t\n']

In [405]:
regex.split("((?<=\(\%\w+)(?=\)))|(\%\w+,)|(\%\w+\s)",assembly)

['\n\t.section\t__TEXT,__text,regular,pure_instructions\n\t.build_version macos, 10, 14\tsdk_version 10, 14\n\t.globl\t_super_duper_print      ## -- Begin function super_duper_print\n\t.p2align\t4, 0x90\n_super_duper_print:                     ## @super_duper_print\n\t.cfi_startproc\n## %bb.0:\n\tpushq\t',
 None,
 None,
 '%rbp\n',
 '\t.cfi_def_cfa_offset 16\n\t.cfi_offset ',
 None,
 '%rbp,',
 None,
 ' -16\n\tmovq\t',
 None,
 '%rsp,',
 None,
 ' ',
 None,
 None,
 '%rbp\n',
 '\t.cfi_def_cfa_register ',
 None,
 None,
 '%rbp\n',
 '\tsubq\t$16, ',
 None,
 None,
 '%rsp\n',
 '\tmovq\t',
 None,
 '%rdi,',
 None,
 ' -8(%rbp',
 '',
 None,
 None,
 ')\n\tmovq\t-8(%rbp',
 '',
 None,
 None,
 '), ',
 None,
 None,
 '%rsi\n',
 '\tleaq\tL_.str(%rip',
 '',
 None,
 None,
 '), ',
 None,
 None,
 '%rdi\n',
 '\tmovb\t$0, ',
 None,
 None,
 '%al\n',
 '\tcallq\t_printf\n\tmovl\t',
 None,
 '%eax,',
 None,
 ' -12(%rbp',
 '',
 None,
 None,
 ')         ## 4-byte Spill\n\taddq\t$16, ',
 None,
 None,
 '%rsp\n',
 '\tpopq

In [404]:
[i for i in regex.split("(?<=\()|(?=\))|(?<=\%\w+,)|(\%\w+\s)|\s",s,) if i is not None and i is not '']

['.direective_here_that_is_not_important_',
 '.second_direcitve_here',
 'movq',
 '-8(',
 '%rax',
 '),',
 '%rsp\t',
 'mov',
 '-16(',
 '$rsp',
 '),',
 '%rax\t',
 '.third',
 'directive',
 'here']

In [402]:
regex.split("(?<=\()|(?=\))|(?<=\%\w+,)|(?=\%\w+\s)",s,)

['.direective_here_that_is_not_important_\t\t\n .second_direcitve_here\t\t\n movq -8(',
 '%rax',
 '), ',
 '%rsp\t\n mov -16(',
 '$rsp',
 '), ',
 '%rax\t\n .third directive here\t\t\n']

In [51]:
regexp_tokenize(s, "(\%\w+)|(,)|(\s)", gaps = True)

['%rax', ',', ' ', '%rsp', '\t']

In [75]:
re.sub("\..*\n", "", s)

'  movq %rax, %rsp\t\n mov $rsp, %rax\t\n '

In [159]:
clean_regexp_string = r"(\..*\n)|(\#.*)"
register_regex = r"(\%\w+)|(,)|(\s)"
instructions_regex = "|".join(set(cleaned_instructions_lower))


In [160]:
register_regex + "|" + instructions_regex

'(\\%\\w+)|(,)|(\\s)|sete|vphsubsw|vpermilps|clflush|blendpd|pushfd|vsqrtpd|vmovapd|vfmsub231ss|vcvtpd2ps|movlhps|jpo|vmovntpd|vpunpckldq|fucompp|vaesenclast|dpps|unpcklpd|popfd|vpmovsxwd|pmuludq|pcmpistrm|vpcmpistri|fild|vpinsrb|punpckhbw|vcvtudq2ps|vbroadcastss|vfmadd231ps|pand|xorpd|vpsllvw|sha256rnds2|blendvpd|vpmullq|vcvtsd2usi|vfixupimmpd|jns|vplzcntd|fsubr|vcvttps2qq|vfnmadd132ss|vsubsd|frndint|vfmaddsub231pd|fldl2t|or|fld1|vpminuw|pminsd|test|kaddw|vbroadcasti64x4|knotb|vpermi2q|xrstor64|int|vfmsubadd231ps|vfmsubadd213pd|vextracti32x4|cmps|lar|movdqa|cwd|vpminub|jrcxz|bndcu|vfnmsub231pd|vpscatterdq|vprold|cmppd|kxnorq|vpextrb|into|kxorq|vmaxps|subss|rcr|psubd|pmaxuw|setng|pabsw|vmovntdqa|fcom|vpcmpistrm|fdivrp|vpsrlw|ftst|vroundsd|cmovle|vaddsd|vcmpss|vbroadcasti32x2|subps|paddusw|vmovntps|aesimc|movsd|cvtps2dq|maxss|extractps|jcxz|movntdqa|vprolvd|bzhi|cld|movzx|vtestps|sal|jpe|cvtdq2pd|lzcnt|andnps|cvtps2pd|smsw|vfpclasssd|lodsw|vpcompressd|fbld|cvtpd2dq|vblendpd|andnpd|vmovd

In [175]:
regexp_tokenize(assembly, instructions_regex, gaps = False)

['lar',
 'in',
 'str',
 'int',
 'in',
 'int',
 'int',
 'int',
 'push',
 'mov',
 'sub',
 'mov',
 'mov',
 'lea',
 'str',
 'mov',
 'call',
 'int',
 'mov',
 'add',
 'pop',
 'ret',
 'in',
 'in',
 'in',
 'in',
 'in',
 'push',
 'mov',
 'sub',
 'mov',
 'lea',
 'str',
 'call',
 'int',
 'mov',
 'add',
 'pop',
 'ret',
 'str',
 'in',
 'str',
 'in',
 'str',
 'str',
 'str',
 'str',
 'les',
 'sub']

In [177]:
def tokenize_assembly(assembly_string: str): 
    split_string = instructions_regex + "|"  + register_regex 
    
    cleaned_assembly = re.sub(clean_regexp_string, "", assembly_string)
    cleaned_assembly = re.sub("\s+", " ", cleaned_assembly)
    print(f"cleaned assembly is {cleaned_assembly}")
    split = regexp_tokenize(cleaned_assembly, split_string)
    result = [s for s in split if s not in [" ", ""]]
    return result
    
    
    

In [199]:
s = "movq %rsp, %rbp\n movzd -8(%rax), %rbp\n _moving_is_the_function_name: \n"

In [358]:
p = r"(?<=mov)"
p = r"(?<=(?<!_)mov)"
p = '?<=(?<!_)mov'
p = '?<!(?<=mov)_'
p = "(?<!_)(?<=mov)"
p = '(?<=(?<!_\w+)mov)'

#p = '(mov)(?<!_)'

In [313]:
regexp_tokenize(s, p, gaps = True)

['mov',
 'q %rsp, %rbp\n mov',
 'zd -8(%rax), %rbp\n _moving_is_the_function_name: \n']

In [315]:
regexp_tokenize(s, instructions_regex, gaps = True)

['mov',
 'q',
 ' %rsp, %rbp\n mov',
 'zd -8(%rax), %rbp\n _movin',
 'g_is_the_function_name: \n']

In [316]:
instructions_regex

'(?<=(?<!_)sete)|(?<=(?<!_)vphsubsw)|(?<=(?<!_)vpermilps)|(?<=(?<!_)clflush)|(?<=(?<!_)blendpd)|(?<=(?<!_)pushfd)|(?<=(?<!_)vsqrtpd)|(?<=(?<!_)vmovapd)|(?<=(?<!_)vfmsub231ss)|(?<=(?<!_)vcvtpd2ps)|(?<=(?<!_)movlhps)|(?<=(?<!_)jpo)|(?<=(?<!_)vmovntpd)|(?<=(?<!_)vpunpckldq)|(?<=(?<!_)fucompp)|(?<=(?<!_)vaesenclast)|(?<=(?<!_)dpps)|(?<=(?<!_)unpcklpd)|(?<=(?<!_)popfd)|(?<=(?<!_)vpmovsxwd)|(?<=(?<!_)pmuludq)|(?<=(?<!_)pcmpistrm)|(?<=(?<!_)vpcmpistri)|(?<=(?<!_)fild)|(?<=(?<!_)vpinsrb)|(?<=(?<!_)punpckhbw)|(?<=(?<!_)vcvtudq2ps)|(?<=(?<!_)vbroadcastss)|(?<=(?<!_)vfmadd231ps)|(?<=(?<!_)pand)|(?<=(?<!_)xorpd)|(?<=(?<!_)vpsllvw)|(?<=(?<!_)sha256rnds2)|(?<=(?<!_)blendvpd)|(?<=(?<!_)vpmullq)|(?<=(?<!_)vcvtsd2usi)|(?<=(?<!_)vfixupimmpd)|(?<=(?<!_)jns)|(?<=(?<!_)vplzcntd)|(?<=(?<!_)fsubr)|(?<=(?<!_)vcvttps2qq)|(?<=(?<!_)vfnmadd132ss)|(?<=(?<!_)vsubsd)|(?<=(?<!_)frndint)|(?<=(?<!_)vfmaddsub231pd)|(?<=(?<!_)fldl2t)|(?<=(?<!_)or)|(?<=(?<!_)fld1)|(?<=(?<!_)vpminuw)|(?<=(?<!_)pminsd)|(?<=(?<!_)test)|(?<=

In [348]:
"|".join(["((?<=(?<!_))" + s + ")" for s in ['mov', 'sub', 'in']])

'((?<=(?<!_))mov)|((?<=(?<!_))sub)|((?<=(?<!_))in)'

In [347]:
list(set(cleaned_instructions_lower))[730:740]

['stos',
 'setl',
 'fprem1',
 'cmpsd',
 'phsubw',
 'in',
 'cmovnle',
 'popad',
 'vpaddsb',
 'fisub']

In [345]:
toy_regex = "|".join(["(?<=(?<!_)" + instruction + ")" for instruction in list(set(cleaned_instructions_lower))[730:740]])







In [365]:
toy_regex = "|".join(["(?<=(?<!(_\w*))" + s + ")" for s in ['mov', 'sub', 'in']])

In [357]:
import regex

In [364]:
p = '(?<=(?<!(_\w*))mov)'
regex.split(p, s)

['mov',
 None,
 'q %rsp, %rbp\n mov',
 None,
 'zd -8(%rax), %rbp\n _moving_is_the_function_name: \n']

In [367]:
regex.split(toy_regex, s)

['mov',
 None,
 None,
 None,
 'q %rsp, %rbp\n mov',
 None,
 None,
 None,
 'zd -8(%rax), %rbp\n _moving_is_the_function_name: \n']

In [368]:
instructions_regex = "|".join(["(?<=(?<!(_\w*))" + instruction + ")" for instruction in set(cleaned_instructions_lower)])

In [371]:
instructions_regex_out = regex.split(instructions_regex, s)
list(filter(None, instructions_regex_out))

['mov',
 'q',
 ' %rsp, %rbp\n mov',
 'zd -8(%rax), %rbp\n _moving_is_the_function_name: \n']

In [354]:
regexp_tokenize(s, toy_regex, gaps = True)

['mov',
 'q %rsp, %rbp\n ',
 'mov',
 'zd -8(%rax), %rbp\n _mov',
 'in',
 'g_is_the_function_name: \n']

In [314]:
instructions_regex = "|".join(["(?<=(?<!_)" + instruction + ")" for instruction in set(cleaned_instructions_lower)])

In [264]:
instructions_regex[:50]

'(?<=(?<!_))sete|(?<=(?<!_))vphsubsw|(?<=(?<!_))vpe'

In [265]:
l = tokenize_assembly(assembly)
l

cleaned assembly is  _super_duper_print: pushq %rbp movq %rsp, %rbp subq $16, %rsp movq %rdi, -8(%rbp) movq -8(%rbp), %rsi leaq L_ movb $0, %al callq _printf movl %eax, -12(%rbp) addq $16, %rsp popq %rbp retq _main: pushq %rbp movq %rsp, %rbp subq $16, %rsp movl $0, -4(%rbp) leaq L_ callq _super_duper_print movl $1, %eax addq $16, %rsp popq %rbp retq L_ " L_ " 


[('', '', ' '),
 ('', '', ''),
 ('', '', ' '),
 ('', '', ''),
 ('', '', ' '),
 ('%rbp', '', ''),
 ('', '', ' '),
 ('', '', ''),
 ('', '', ' '),
 ('%rsp', '', ''),
 ('', ',', ''),
 ('', '', ' '),
 ('%rbp', '', ''),
 ('', '', ' '),
 ('', '', ''),
 ('', '', ' '),
 ('', ',', ''),
 ('', '', ' '),
 ('%rsp', '', ''),
 ('', '', ' '),
 ('', '', ''),
 ('', '', ' '),
 ('%rdi', '', ''),
 ('', ',', ''),
 ('', '', ' '),
 ('%rbp', '', ''),
 ('', '', ' '),
 ('', '', ''),
 ('', '', ' '),
 ('%rbp', '', ''),
 ('', ',', ''),
 ('', '', ' '),
 ('%rsi', '', ''),
 ('', '', ' '),
 ('', '', ''),
 ('', '', ' '),
 ('', '', ' '),
 ('', '', ''),
 ('', '', ' '),
 ('', ',', ''),
 ('', '', ' '),
 ('%al', '', ''),
 ('', '', ' '),
 ('', '', ''),
 ('', '', ' '),
 ('', '', ''),
 ('', '', ' '),
 ('', '', ''),
 ('', '', ' '),
 ('%eax', '', ''),
 ('', ',', ''),
 ('', '', ' '),
 ('%rbp', '', ''),
 ('', '', ' '),
 ('', '', ''),
 ('', '', ' '),
 ('', ',', ''),
 ('', '', ' '),
 ('%rsp', '', ''),
 ('', '', ' '),
 ('', '', ''),
 (

In [132]:
b = [s for s in l if s != None]
b = [s for s in b if s not in [" ", ""]]

In [133]:
b

['_super_duper_print:',
 'pushq',
 '%rbp',
 'movq',
 '%rsp',
 ',',
 '%rbp',
 'subq',
 '$16',
 ',',
 '%rsp',
 'movq',
 '%rdi',
 ',',
 '-8(',
 '%rbp',
 ')',
 'movq',
 '-8(',
 '%rbp',
 ')',
 ',',
 '%rsi',
 'leaq',
 'L_',
 'movb',
 '$0',
 ',',
 '%al',
 'callq',
 '_printf',
 'movl',
 '%eax',
 ',',
 '-12(',
 '%rbp',
 ')',
 'addq',
 '$16',
 ',',
 '%rsp',
 'popq',
 '%rbp',
 'retq',
 '_main:',
 'pushq',
 '%rbp',
 'movq',
 '%rsp',
 ',',
 '%rbp',
 'subq',
 '$16',
 ',',
 '%rsp',
 'movl',
 '$0',
 ',',
 '-4(',
 '%rbp',
 ')',
 'leaq',
 'L_',
 'callq',
 '_super_duper_print',
 'movl',
 '$1',
 ',',
 '%eax',
 'addq',
 '$16',
 ',',
 '%rsp',
 'popq',
 '%rbp',
 'retq',
 'L_',
 '"',
 'L_',
 '"']

In [98]:
clean_regexp_string = r"(\..*\n)|(\#.*\n)"
clean_regexp_string = r"\..*\n"

In [111]:
clean_regexp_string = r"(\..*\n)|(\#.*)"
s1 = re.sub(clean_regexp_string, "", assembly)

print(s1)


				_super_duper_print:                     
	
	pushq	%rbp
			movq	%rsp, %rbp
		subq	$16, %rsp
	movq	%rdi, -8(%rbp)
	movq	-8(%rbp), %rsi
	leaq	L_	movb	$0, %al
	callq	_printf
	movl	%eax, -12(%rbp)         
	addq	$16, %rsp
	popq	%rbp
	retq
	                                        
		_main:                                  
	
	pushq	%rbp
			movq	%rsp, %rbp
		subq	$16, %rsp
	movl	$0, -4(%rbp)
	leaq	L_	callq	_super_duper_print
	movl	$1, %eax
	addq	$16, %rsp
	popq	%rbp
	retq
	                                        
	L_	"

L_	"






In [112]:
clean_regexp_string = r"\..*\n"
s2 = re.sub(clean_regexp_string, "", assembly)

print(s2)


				_super_duper_print:                     ## @super_duper_print
	## %bb	pushq	%rbp
			movq	%rsp, %rbp
		subq	$16, %rsp
	movq	%rdi, -8(%rbp)
	movq	-8(%rbp), %rsi
	leaq	L_	movb	$0, %al
	callq	_printf
	movl	%eax, -12(%rbp)         ## 4-byte Spill
	addq	$16, %rsp
	popq	%rbp
	retq
	                                        ## -- End function
		_main:                                  ## @main
	## %bb	pushq	%rbp
			movq	%rsp, %rbp
		subq	$16, %rsp
	movl	$0, -4(%rbp)
	leaq	L_	callq	_super_duper_print
	movl	$1, %eax
	addq	$16, %rsp
	popq	%rbp
	retq
	                                        ## -- End function
	L_	"

L_	"






In [89]:
l[100:150]

['',
 None,
 None,
 ' ',
 '##',
 None,
 None,
 ' ',
 '@super_duper_print',
 None,
 None,
 '\n',
 '',
 None,
 None,
 '\t',
 '##',
 None,
 None,
 ' ',
 '',
 '%bb',
 None,
 None,
 '',
 None,
 None,
 '\t',
 'pushq',
 None,
 None,
 '\t',
 '',
 '%rbp',
 None,
 None,
 '',
 None,
 None,
 '\n',
 '',
 None,
 None,
 '\t',
 '',
 None,
 None,
 '\t',
 '',
 None]

In [ ]:
# def split_regex(instructions: List[str], register_regexp: str = r"(\%\w+)|(,)|(\s)", instruction_prefix: str = r"(?<=(?<!(_\w*))"): 
#     indiv_regexp = [instruction_prefix + instruction + ")" for instruction in instructions]
#     joined_instruction_regexp = "|".join(indiv_regexp)
#     split_regexp = joined_instruction_regexp + register_regexp
#     return split_regexp
    
# def split_regex(instructions: List[str], register_regexp: str = r"(?<=\()|(?=\))|(?<=\%\w+,)|(\%\w+\s)|\s", instruction_prefix: str = r"(?<=(?<!(_\w*))"): 
#     indiv_regexp = [instruction_prefix + instruction + ")" for instruction in instructions]
#     joined_instruction_regexp = "|".join(indiv_regexp)
#     split_regexp = joined_instruction_regexp + register_regexp
#     return split_regexp
    
# def split_regex(instructions: List[str], 
#                 register_regexp: str = r"(?<=\%\w{2,5})(?=,)|(?<=\s)(?=\%\w{2,5})|(?<=\%\w{2,5})(?=\s)|(?<=\%\w{2,5})(?=\))|(?<=\()(?=\%)", 
#                 instruction_prefix: str = r"(?<=(?<!(_\w*))"): 
#     indiv_regexp = [instruction_prefix + instruction + ")" for instruction in instructions]
#     joined_instruction_regexp = "|".join(indiv_regexp)
#     split_regexp = joined_instruction_regexp + register_regexp
#     return split_regexp
    
